In [1]:
import sys
import os

def is_colab_env():
    return "google.colab" in sys.modules

def mount_google_drive(drive_dir="/content/drive/", repo_dir="MyDrive/repositories/deepfake-detection"):
    # mount google drive
    from google.colab import drive
    drive.mount(drive_dir)

    # change to correct working directory
    import os
    os.chdir(f"{drive_dir}{repo_dir}")
    print(os.listdir()) # verify content

def resolve_path(levels_deep=3):
    if is_colab_env():
        mount_google_drive()
    else:
        # Get the directory of the current script
        current_dir = os.path.dirname(os.path.abspath('__file__'))

        # Construct the path to the parent directory
        for i in range(levels_deep):
            current_dir = os.path.dirname(current_dir)

        # Add the parent directory to sys.path
        sys.path.append(current_dir)
        print(sys.path)

resolve_path()

['/home/FYP/tanj0303/.conda/envs/df-env/lib/python310.zip', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/lib-dynload', '', '/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/site-packages', '/home/FYP/tanj0303']


In [2]:
# import local config
import config

In [3]:
# import library dependencies
import io
from itertools import islice
import matplotlib.pyplot as plt
import numpy as np

# image processing
import cv2
from PIL import Image

In [4]:
# pytorch
import torch
import pytorch_lightning as L

In [5]:
# import local dependencies
# from src.adapters.datasets.wilddeepfake import WildDeepfakeDataModule
from src.adapters.datasets.sida import SidADataModule
from src.models.freqnet import LitFreqNet

In [6]:
model_id = "frequency_freqnet"
model_checkpoint_dir = f"{config.CHECKPOINTS_DIR}/{model_id}"

In [7]:
from torchvision import transforms

# --- common normalization (ImageNet) ---
imagenet_normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

# --- training transform ---
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),        # resize frame
    transforms.Lambda(lambda img: img.convert("RGB")),  # force RGB
    transforms.RandomHorizontalFlip(),    # flip for augmentation
    transforms.ColorJitter(               # optional: color variation
        brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1
    ),
    transforms.ToTensor(),
    imagenet_normalize,
])

# --- validation transform ---
val_transform = transforms.Compose([
    transforms.Resize((256, 256)),  # deterministic resize
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    imagenet_normalize,
])

# --- test transform (usually same as val) ---
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Lambda(lambda img: img.convert("RGB")),
    transforms.ToTensor(),
    imagenet_normalize,
])

transforms = {
    "train": train_transform,
    "val": val_transform,
    "test": test_transform
}

In [8]:
# Set seeds for reproducibility
seed = config.SEED

torch.manual_seed(seed)
np.random.seed(seed)

# Determine device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [9]:
dataset_name = "xingjunm/WildDeepfake"
max_samples = 994_000  # For quick development, remove for full dataset
batch_size = 16
num_workers = 2
max_epochs = 30

# define datamodule
# wilddeepfake_data_module = WildDeepfakeDataModule(
#     dataset_name=dataset_name,
#     batch_size=batch_size,
#     num_workers=num_workers,
#     max_samples=max_samples,
#     seed=seed,
#     transforms=DEFAULT_DATA_TRANSFORMS,
#     additional_transforms=ela
# )
from src.adapters.datasets.wilddeepfake import load_streaming_dataset, create_data_loaders
datasets = load_streaming_dataset(
    dataset_name,
    max_samples=max_samples,
    seed=seed
)
train_loader, val_loader, test_loader = create_data_loaders(
    datasets,
    batch_size=batch_size,
    num_workers=num_workers,
    transforms=transforms,
    # additional_transforms=ela
)

Loading streaming dataset: xingjunm/WildDeepfake


Resolving data files:   0%|          | 0/963 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/157 [00:00<?, ?it/s]

In [10]:
# define early stopper
early_stop_callback = L.callbacks.EarlyStopping(
    monitor="val_loss",       # metric to track
    patience=3,               # epochs to wait for improvement
    mode="min",               # "min" because we want val_loss to decrease
    verbose=True
)

In [ ]:
# define ligntning checkpoint
best_loss_checkpoint = L.callbacks.ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
)

In [12]:
# define model
deepfake_detector = LitFreqNet()

In [13]:
trainer = L.Trainer(
    devices=1,
    callbacks=[early_stop_callback, best_loss_checkpoint],
    default_root_dir=model_checkpoint_dir,
    log_every_n_steps=10,
    profiler="simple", # track time taken
    max_steps= max_epochs * max_samples / batch_size, #(desired_epochs × dataset_size) / batch_size
    # limit_train_batches=1000,   # how many batches per "epoch"
    # limit_val_batches=200,      # how many val batches per "epoch"
  )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [ ]:
# train model
# trainer.fit(deepfake_detector, datamodule=sida_data_module)
trainer.fit(deepfake_detector, train_dataloaders=train_loader, val_dataloaders=val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type              | Params | Mode 
----------------------------------------------------------
0  | model      | FreqNet           | 1.9 M  | train
1  | loss_fn    | BCEWithLogitsLoss | 0      | train
2  | train_acc  | BinaryAccuracy    | 0      | train
3  | val_acc    | BinaryAccuracy    | 0      | train
4  | test_acc   | BinaryAccuracy    | 0      | train
5  | train_prec | BinaryPrecision   | 0      | train
6  | val_prec   | BinaryPrecision   | 0      | train
7  | test_prec  | BinaryPrecision   | 0      | train
8  | train_rec  | BinaryRecall      | 0      | train
9  | val_rec    | BinaryRecall      | 0      | train
10 | test_rec   | BinaryRecall      | 0      | train
11 | train_f1   | BinaryF1Score     | 0      | train
12 | val_f1     | BinaryF1Score     | 0      | train
13 | test_f1    | BinaryF1Score     | 0      | train
14 | train_auc  | BinaryAUROC       | 0      | train
15 | val_auc    | BinaryAUROC       | 0      | tra

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/FYP/tanj0303/.conda/envs/df-env/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:123: Your `IterableDataset` has `__len__` defined. In combination with multi-process data loading (when num_workers > 1), `__len__` could be inaccurate if each worker is not configured independently to avoid having duplicate data.


Training: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
# test dataset on unseen samples
# trainer.test(deepfake_detector, datamodule=sida_data_module)
trainer.test(deepfake_detector, test_loader)

In [ ]:
# view metrics from previous runs
%reload_ext tensorboard
%tensorboard --logdir=$model_checkpoint_dir

In [ ]:

import torch
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version from torch:', torch.version.cuda)
print('Device count:', torch.cuda.device_count())
